# Goal
create a data set for tutorial with john henry for us to do power analysis. it doesn't matter what the data is as long as it hasthe right shape.
the rows should like:
1. 1 row represents one user who's participated in wla at some point
1. Latest year participated
1. How many images the contributed latest year
1. list of images they contributed
1. How many edits they made during that year?
1. How many other images they contributed that year?
1. Image-view-count in that year (for all WLA images)?
1. \[nice to have\] image view count of their most contributed image
1. their user-language preference on commons


## Pipeline steps
1. get images and actors from commons categories, sql
1. generate mediacounts per day, need to multiprocess/redis
1. aggregate mediacounts to year,
1. join image-year and actor
2. aggregate on actor,
1. add actor level columns

In [17]:
import pandas as pd
import os, json
import numpy as np

In [127]:
project_dir = '/data/project/wla/'
share_dir = '/home/paprika/workspace/civilservant-wla-analysis'

In [3]:
media_counts_history_f = os.path.join(project_dir, 'pre-pipeline-2018-analysis', 'wla-mediacounts-by-day-2015-2018.csv')

In [4]:
media_counts_f = os.path.join(project_dir, 'mediacounts-output','wla_mediacounts_2015.csv')
usecols = ['filename','tsv_name','total_transfers']

In [5]:
hist = pd.read_csv(media_counts_history_f, usecols=['filename_unquoted','date','total_transfers'], parse_dates=['date'])

In [6]:
hist=hist.rename(columns={'filename_unquoted':'img_name'})

In [7]:
hist.head()

,img_name,date,total_transfers
0,Products.jpg,2015-01-01,1
1,Dance.png,2015-01-01,1
2,الفريق.jpg,2015-01-01,229
3,خبز.JPG,2015-01-01,206
4,Her.jpg,2015-01-01,53


In [8]:
hist.shape

(16265025, 3)

In [9]:
hist['year'] = hist['date'].apply(lambda d: d.year)

In [10]:
hist['year'].value_counts()

2018    6613716
2017    4552378
2016    2810379
2015    1494413
2019     794139
Name: year, dtype: int64

In [11]:
pre = hist[hist['year'] <=  2016]
post = hist[hist['year'] == 2017]

In [37]:
img_aggregations = {'total_transfers':sum, 'date':len,}

pre_i = pre.groupby('img_name').agg(img_aggregations).reset_index()
post_i = post.groupby('img_name').agg(img_aggregations).reset_index()

In [38]:
user_filename_f = os.path.join(project_dir,'joining-relations','actor_images_years.csv')
user_filename = pd.read_csv(user_filename_f, index_col=0)

In [39]:
user_filename_pre = user_filename[user_filename['year']<=2016]
user_filename_post = user_filename[user_filename['year']==2017]

In [40]:
user_filename_post['year'].max()

2017

In [41]:
pre_user = pre_i.merge(user_filename_pre, on='img_name', how='inner')
post_user = post_i.merge(user_filename_post, on='img_name', how='inner')

In [42]:
pre_user.shape, post_user.shape

((12877, 5), (16674, 5))

In [43]:
pre_user.head(2)

,img_name,total_transfers,date,img_actor,year
0,--_Africa_standing.jpg,10067,441,5389266,2015
1,--ewinosa.jpg,9930,437,5389266,2015


In [90]:
user_agg_rename

{'img_name': 'images_contributed', 'year': 'num_years_competed'}

In [115]:
# the reason for the year subsetting is that year is the year of wla competition
# remember pre reperesents all the image views that were 2016 and earlier

def images_contributed(s):
    return len(set(s))

def years_competed(s):
    return len(set(s))

def total_transfers(s):
    return sum(s)

def latest_year_competed(s):
    return max(s)

user_aggregations = {'total_transfers':total_transfers,
                     'img_name':images_contributed,
                    'year':(years_competed,
                            latest_year_competed),
                    }
user_agg_rename = {'img_name': 'images_contributed', 'year': 'num_years_competed','':'img_actor'}

user_pre = pre_user.groupby('img_actor').agg(user_aggregations).reset_index()
user_pre.columns = user_pre.columns.get_level_values(1)
user_pre = user_pre.rename(columns=user_agg_rename)

user_post = post_user.groupby('img_actor').agg(user_aggregations).reset_index()
user_post.columns = user_post.columns.get_level_values(1)
user_post = user_post.rename(columns=user_agg_rename)

In [116]:
user_pre.shape, user_post.shape

((1736, 5), (2349, 5))

In [117]:
user_post.head(10)

,img_actor,total_transfers,images_contributed,years_competed,latest_year_competed
0,5,7359,22,1,2017
1,13,17823,62,1,2017
2,30,29854,216,1,2017
3,86,11725,5,1,2017
4,89,177249,634,1,2017
5,134,249,1,1,2017
6,157,14150,2,1,2017
7,396,398,1,1,2017
8,402,93,1,1,2017
9,431,1142,13,1,2017


In [118]:
returning = user_pre.merge(user_post, on='img_actor', how='outer', suffixes=("_2016_and_earlier", "_2017"))

In [119]:
returning

,img_actor,total_transfers_2016_and_earlier,images_contributed_2016_and_earlier,years_competed_2016_and_earlier,latest_year_competed_2016_and_earlier,total_transfers_2017,images_contributed_2017,years_competed_2017,latest_year_competed_2017
0,13,821116.0,120.0,3.0,2016.0,17823.0,62.0,1.0,2017.0
1,30,40707.0,17.0,2.0,2015.0,29854.0,216.0,1.0,2017.0
2,62,7284.0,1.0,1.0,2015.0,NaN,NaN,NaN,NaN
3,86,4503.0,1.0,1.0,2014.0,11725.0,5.0,1.0,2017.0
4,125,19740.0,3.0,1.0,2015.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4018,6933779,NaN,NaN,NaN,NaN,5968.0,3.0,1.0,2017.0
4019,6934063,NaN,NaN,NaN,NaN,845.0,4.0,1.0,2017.0
4020,6934545,NaN,NaN,NaN,NaN,794.0,7.0,1.0,2017.0
4021,6935237,NaN,NaN,NaN,NaN,879.0,10.0,1.0,2017.0


In [120]:
didnt_return = returning[(pd.notnull(returning['images_contributed_2016_and_earlier'])) &
         (pd.isnull(returning['images_contributed_2017']))]

In [121]:
first_time = returning[(pd.isnull(returning['images_contributed_2016_and_earlier'])) &
         (pd.notnull(returning['images_contributed_2017']))]

In [122]:
returned = returning[(pd.notnull(returning['images_contributed_2016_and_earlier'])) &
         (pd.notnull(returning['images_contributed_2017']))]

In [123]:
contributed_2016_and_earlier = returning[(pd.notnull(returning['images_contributed_2016_and_earlier']))]

In [124]:
returning["didnt_return"] = returning['images_contributed_2016_and_earlier'].notnull()  &  returning['images_contributed_2017'].isnull()
returning["return"] = returning['images_contributed_2016_and_earlier'].notnull()  &  returning['images_contributed_2017'].notnull()
returning['first_time'] = returning['images_contributed_2016_and_earlier'].isnull() & \
         returning['images_contributed_2017'].notnull()

In [125]:
returning.head()

,img_actor,total_transfers_2016_and_earlier,images_contributed_2016_and_earlier,years_competed_2016_and_earlier,latest_year_competed_2016_and_earlier,total_transfers_2017,images_contributed_2017,years_competed_2017,latest_year_competed_2017,didnt_return,return,first_time
0,13,821116.0,120.0,3.0,2016.0,17823.0,62.0,1.0,2017.0,False,True,False
1,30,40707.0,17.0,2.0,2015.0,29854.0,216.0,1.0,2017.0,False,True,False
2,62,7284.0,1.0,1.0,2015.0,NaN,NaN,NaN,NaN,True,False,False
3,86,4503.0,1.0,1.0,2014.0,11725.0,5.0,1.0,2017.0,False,True,False
4,125,19740.0,3.0,1.0,2015.0,NaN,NaN,NaN,NaN,True,False,False


In [130]:
out_f = os.path.join(project_dir, 'pre-pipeline-2018-analysis', 'power-analysis-2016-vs-2017.csv')
share_f = os.path.join(share_dir, 'data', 'power-analysis-2016-vs-2017.csv')
returning.to_csv(out_f, index=False)
returning.to_csv(share_f, index=False)

In [138]:
user_filename.groupby(['year']).agg({'img_actor':lambda s: len(set(s))})

,img_actor
year,
2014,845
2015,725
2016,827
2017,2440
2019,1299


In [142]:
#number of unique users
user_filename.groupby('img_actor').agg(len).agg(len)

img_name    5890
year        5890
dtype: int64

In [146]:
len(set(user_filename['img_actor'].values))

5890

In [110]:
didnt_return.shape, first_time.shape, returned.shape, contributed_2016_and_earlier.shape

((1674, 5), (2287, 5), (62, 5), (1736, 5))

In [41]:
post[post['img_name']=='Traditional_Dress_sales.jpg']['year'].min()

2017

In [ ]:
user_pre

In [15]:
media_counts = pd.read_csv(media_counts_f,usecols=usecols)

In [3]:
media_counts.columns

Index(['filename', 'total_transfers', 'tsv_name'], dtype='object')

In [4]:
media_counts['img_name'] = media_counts['filename'].apply(lambda n: n.split('/')[-1])

In [5]:
media_counts.head()

,filename,total_transfers,tsv_name,img_name
0,/wikibooks/bg/9/91/Products.jpg,2,mediacounts.2016-01-01.v00.tsv.bz2,Products.jpg
1,/wikipedia/ar/9/91/Products.jpg,1,mediacounts.2016-01-01.v00.tsv.bz2,Products.jpg
2,/wikipedia/ar/9/9b/Her.jpg,61,mediacounts.2016-01-01.v00.tsv.bz2,Her.jpg
3,/wikipedia/ar/d/d3/Om_ali.jpg,11,mediacounts.2016-01-01.v00.tsv.bz2,Om_ali.jpg
4,/wikipedia/bs/f/fc/Oasis.jpg,2,mediacounts.2016-01-01.v00.tsv.bz2,Oasis.jpg


In [6]:
#transform to image-year
#tsv_name should be at least
media_counts_i = media_counts.groupby('img_name').agg({'total_transfers':sum, 'tsv_name':len}).reset_index()

In [7]:
media_counts_i.tail()

,img_name,total_transfers,tsv_name
11541,Zucchini_1.jpg,1289,333
11542,Zulu_Dancer.jpg,223,28
11543,Zulu_Traditional_attire.jpg,2872,364
11544,Zulu_X(1).jpg,84,5
11545,Zulu_attire.jpg,1485,354


In [8]:
user_filename.head()

,img_actor,img_name,year
0,47413,Wiki_Loves_Africa.pdf,2014
1,4297937,Home_Chef_-_African_food_and_cuisine.jpg,2014
2,3539708,سوق_بمدينة_المنستير_التونسية_1.JPG,2014
3,3539708,سوق_بمدينة_المنستير_التونسية_2.JPG,2014
4,3539708,سوق_بمدينة_المنستير_التونسية_3.JPG,2014


In [9]:
image_user = media_counts_i.merge(user_filename, on='img_name', how='left')

In [10]:
image_user.tail()

,img_name,total_transfers,tsv_name,img_actor,year
11527,Zucchini_1.jpg,1289,333,3539708,2014
11528,Zulu_Dancer.jpg,223,28,2382602,2016
11529,Zulu_Traditional_attire.jpg,2872,364,5560173,2015
11530,Zulu_X(1).jpg,84,5,6331884,2016
11531,Zulu_attire.jpg,1485,354,5524354,2015


In [11]:
len(set(pd.Series([2014,2015])))

2

In [12]:
# 1. Latest year participated
# 1. How many images the contributed latest year
# 1. list of images they contributed
# 1. Image-view-count in that year (for all WLA images)?
# 1. \[nice to have\] image view count of their most contributed image # would need to aggregate on multiple columns

def latest_year_participated(series):
    return max(series)

def num_years_participated(series):
    return len(set(series))

def contributed_images_json(series):
    return json.dumps(list(series.values))
               
def num_views_all_wla_images_ever(series):
    return series.sum()

def num_images_contributed_ever(series):
    return len(set(series))
               

agg_d = {}
agg_d['img_name'] = contributed_images_json, num_images_contributed_ever
agg_d['total_transfers'] = num_views_all_wla_images_ever
agg_d['year'] = latest_year_participated, num_years_participated

user_df = image_user.groupby('img_actor').agg(agg_d)
user_df.columns = user_df.columns.get_level_values(1)
user_df=user_df.reset_index()

In [13]:
user_df.head()

,img_actor,contributed_images_json,num_images_contributed_ever,num_views_all_wla_images_ever,latest_year_participated,num_years_participated
0,13,"[""AFRICAN_-_AMERICAN.JPG"", ""ARMLESS_KANGA.JPG""...",111,293570,2016,3
1,30,"[""African_shades.jpg"", ""Les_Amazones_d'Afrique...",100,54071,2016,3
2,62,"[""Robe_khabyel.jpg""]",1,5286,2015,1
3,89,"[""Big_Dreams.jpg"", ""Photojournalist.jpg""]",2,627,2017,1
4,125,"[""Collier_MHNT_ETH_2012_23_108.jpg"", ""Collier_...",3,19740,2015,1


In [14]:
user_df.shape

(1682, 6)

In [15]:
user_df.iloc[0]["contributed_images_json"]

'["AFRICAN_-_AMERICAN.JPG", "ARMLESS_KANGA.JPG", "A_Young_couple.JPG", "Acha_soup.jpg", "Africa_at_Church_03.JPG", "Art_work_from_Essaouira.jpg", "Asili_ya_Kitenge.jpg", "Asili_ya_Mwafrika.jpg", "Asilia_ya_Mwafrika.jpg", "Berrad.jpg", "Berrad_sur_feu.jpg", "Broderie_1.JPG", "Chania_15.jpg", "Chaussure_femme_traditionnelle_45.JPG", "Chungu.jpg", "Confection_des_habits_tradionnels_1.JPG", "Culture_Fashion_Tunisia_10.JPG", "Culture_Fashion_Tunisia_14.JPG", "Culture_Fashion_Tunisia_35.JPG", "DGdanse08.JPG", "DGdanseLali.JPG", "DGdanselali04.JPG", "DGdansesecrete04.JPG", "Dizaini.jpg", "Dizaini_ya_kitenge.jpg", "EMPIRE_KANGA.JPG", "EgyptianFruits_PhotoByWaelNawara.jpg", "EgyptianTurkeyAndRiceBelKhalta_PhotoByWaelNawara.jpg", "Egyptian_plate_001.jpg", "Egyptian_plate_002.jpg", "Egyptian_plate_004.jpg", "Egyptian_plate_005.jpg", "Egyptian_plate_006.jpg", "Egyptian_plate_007.jpg", "Egyptian_plate_008.jpg", "Egyptian_plate_010.jpg", "Egyptian_plate_012.jpg", "Egyptian_plate_013.jpg", "Egyptian_

# user -level column adds

In [16]:
# 1. How many edits they made during that year
# 1. How many other images they contributed that year  image/ img_timestamp
# 1. their user-language preference on commons

In [17]:
import civilservant

In [33]:
from civilservant.wikipedia.connections.database import make_wmf_con
from civilservant.wikipedia.queries.revisions import get_timestamps_within_range
from civilservant.wikipedia.queries.users import get_user_name_id_from_actor

Outcome Variables
1) binary whether or not they entered the competition and 
2) how many photos they entered.

In [31]:
wmf_db = make_wmf_con()

In [34]:
def get_user_name_id(nameid, actor_id):
    user_df = get_user_name_id_from_actor('commons', actor_id, wmf_db)
    return user_df.iloc[0][nameid]

In [37]:
user_df = user_df[:10]

In [38]:
user_df['user_name'] = user_df.apply(lambda row: get_user_name_id('user_name', row['img_actor']), axis=1)

/home/paprika/.local/share/virtualenvs/civilservant-wla-wKX6CekY/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
user_df['user_id'] = user_df.apply(lambda row: get_user_name_id('user_id', row['img_actor']), axis=1)

/home/paprika/.local/share/virtualenvs/civilservant-wla-wKX6CekY/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
user_df.head()

,img_actor,contributed_images_json,num_images_contributed_ever,num_views_all_wla_images_ever,latest_year_participated,num_years_participated,user_name,user_id
0,13,"[""AFRICAN_-_AMERICAN.JPG"", ""ARMLESS_KANGA.JPG""...",111,293570,2016,3,b'SteinsplitterBot',3714013
1,30,"[""African_shades.jpg"", ""Les_Amazones_d'Afrique...",100,54071,2016,3,b'Thesupermat',61609
2,62,"[""Robe_khabyel.jpg""]",1,5286,2015,1,b'Secondarywaltz',222859
3,89,"[""Big_Dreams.jpg"", ""Photojournalist.jpg""]",2,627,2017,1,b'Embedded Data Bot',6325156
4,125,"[""Collier_MHNT_ETH_2012_23_108.jpg"", ""Collier_...",3,19740,2015,1,b'Archaeodontosaurus',591251


# Outcome variables

1. binary whether or not they entered the competition and 
2. how many photos they entered.

In [43]:
user_df

,img_actor,contributed_images_json,num_images_contributed_ever,num_views_all_wla_images_ever,latest_year_participated,num_years_participated,user_name,user_id
0,13,"[""AFRICAN_-_AMERICAN.JPG"", ""ARMLESS_KANGA.JPG""...",111,293570,2016,3,b'SteinsplitterBot',3714013
1,30,"[""African_shades.jpg"", ""Les_Amazones_d'Afrique...",100,54071,2016,3,b'Thesupermat',61609
2,62,"[""Robe_khabyel.jpg""]",1,5286,2015,1,b'Secondarywaltz',222859
3,89,"[""Big_Dreams.jpg"", ""Photojournalist.jpg""]",2,627,2017,1,b'Embedded Data Bot',6325156
4,125,"[""Collier_MHNT_ETH_2012_23_108.jpg"", ""Collier_...",3,19740,2015,1,b'Archaeodontosaurus',591251
5,157,"[""AMATA.jpg"", ""BALLS_OF_BANKU.jpg"", ""Baleng_2....",57,1660246,2016,3,b'Ji-Elle',30885
6,662,"[""Tunisian_brown_couscous_with_vegetables.JPG""]",1,1170731,2014,1,b'Takeaway',335606
7,979,"[""Condiments.jpg"", ""Escargot2.JPG"", ""Escargot3...",9,4945,2014,1,b'Zenman',219198
8,1328,"[""WOOD.jpg""]",1,1543,2015,1,b'Amada44',92460
9,1470,"[""Dried_Snoek_near_Grotto_Bay.jpg"", ""Mielies_o...",7,277500,2014,1,b'Ossewa',256347
